In [1]:
import spacy
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
#from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
train['word_count'] = train['text'].apply(lambda x: len(str(x).split(" ")))
train['char_count'] = train['text'].str.len()
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['text'].apply(lambda x: avg_word(x))

from nltk.corpus import stopwords
stop = stopwords.words('english')

train['stopwords'] = train['text'].apply(lambda x: len([x for x in x.split() if x in stop]))

train['hastags'] = train['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
train['numerics'] = train['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

train.head()

,ID,text,target,word_count,char_count,avg_word,stopwords,hastags,numerics
0,train_0,The bitcoin halving is cancelled due to,1,7,39,4.714286,2,0,0
1,train_1,MercyOfAllah In good times wrapped in its gran...,0,20,117,4.900000,7,0,0
2,train_2,266 Days No Digital India No Murder of e learn...,1,20,93,3.700000,2,0,1
3,train_3,India is likely to run out of the remaining RN...,1,23,112,3.913043,11,0,0
4,train_4,In these tough times the best way to grow is t...,0,26,121,3.692308,11,0,0


In [6]:
test['word_count'] = test['text'].apply(lambda x: len(str(x).split(" ")))
test['char_count'] = test['text'].str.len()
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

test['avg_word'] = test['text'].apply(lambda x: avg_word(x))

from nltk.corpus import stopwords
stop = stopwords.words('english')

test['stopwords'] = test['text'].apply(lambda x: len([x for x in x.split() if x in stop]))

test['hastags'] = test['text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
test['numerics'] = test['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

test.head()

,ID,text,word_count,char_count,avg_word,stopwords,hastags,numerics
0,test_0,rinaldi According to the new Italian governmen...,18,128,6.166667,5,0,0
1,test_1,In 2017 a Palestinian judge banned divorce dur...,18,113,5.333333,6,0,1
2,test_2,Why is explained in the video take a look,10,42,3.666667,4,0,0
3,test_3,Ed Davey fasting for Ramadan No contest,7,39,4.714286,1,0,0
4,test_4,Is Doja Cat good or do you just miss Nicki Minaj,11,48,3.454545,4,0,0


In [7]:
train['text'] = train['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
stop = stopwords.words('english')
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [8]:
test['text'] = test['text'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
stop = stopwords.words('english')
test['text'] = test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [9]:
freq = pd.Series(' '.join(train['text']).split()).value_counts()[:10]
freq = list(freq.index)
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['text'].head()
        

0                        bitcoin halving cancelled due
1    MercyOfAllah In good times wrapped granular de...
2    266 Days No Digital India No Murder e learning...
3    India likely run remaining RNA kits essential ...
4    In tough times best way grow learn case teach ...
Name: text, dtype: object

In [10]:
freq = pd.Series(' '.join(test['text']).split()).value_counts()[:10]
freq = list(freq.index)
test['text'] = test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['text'].head()
        

0    rinaldi According new Italian government regul...
1    In 2017 Palestinian judge banned divorce make ...
2                        Why explained video take look
3                          Ed Davey fasting No contest
4                    Is Doja Cat good miss Nicki Minaj
Name: text, dtype: object

In [11]:
#rare words removal
freq = pd.Series(' '.join(train['text']).split()).value_counts()[-10:]
freq = list(freq.index)
train['text'] = train['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train['text'].head()

0                        bitcoin halving cancelled due
1    MercyOfAllah In good times wrapped granular de...
2    266 Days No Digital India No Murder e learning...
3    India likely run remaining RNA kits essential ...
4    In tough times best way grow learn case teach ...
Name: text, dtype: object

In [12]:
#rare words removal
freq = pd.Series(' '.join(test['text']).split()).value_counts()[-10:]
freq = list(freq.index)
test['text'] = test['text'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
test['text'].head()

0    rinaldi According new Italian government regul...
1    In 2017 Palestinian judge banned divorce make ...
2                        Why explained video take look
3                          Ed Davey fasting No contest
4                    Is Doja Cat good miss Nicki Minaj
Name: text, dtype: object

In [13]:
from textblob import TextBlob
train['text'][:5].apply(lambda x: str(TextBlob(x).correct()))
test['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    rinaldi According new Italian government regul...
1    In 2017 Palestinian judge banner divorce make ...
2                        Why explained video take look
3                            D Have fasting To contest
4                      Is Sofa At good miss Sick Final
Name: text, dtype: object

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def data_text_preprocess(total_text, ind, col):
    # Remove int values from text data as that might not be imp
    if type(total_text) is not int:
        string = ""
        # replacing all special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', str(total_text))
        # replacing multiple spaces with single space
        total_text = re.sub('\s+',' ', str(total_text))
        # bring whole text to same lower-case scale.
        total_text = total_text.lower()
        
        for word in total_text.split():
        # if the word is a not a stop word then retain that word from text
            if not word in stop_words:
                string += word + " "
        
        train[col][ind] = string

In [16]:
for index, row in test.iterrows():
    if type(row['text']) is str:
        data_text_preprocess(row['text'], index, 'text')

# Below code will take some time because its huge text (took 4 minute on my 16 GB RAM system), so run it and have a cup of coffee :)
for index, row in train.iterrows():
    if type(row['text']) is str:
        data_text_preprocess(row['text'], index, 'text')
        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# building a CountVectorizer with all the words that occured minimum 3 times in train data
text_vectorizer = CountVectorizer(min_df=3)
train_text_feature_onehotCoding = text_vectorizer.fit_transform(train['text'])
test_text_feature_onehotCoding = text_vectorizer.transform(test['text'])
# getting all the feature names (words)
train_text_features= text_vectorizer.get_feature_names()

# train_text_feature_onehotCoding.sum(axis=0).A1 will sum every row and returns (1*number of features) vector
train_text_fea_counts = train_text_feature_onehotCoding.sum(axis=0).A1

# zip(list(text_features),text_fea_counts) will zip a word with its number of times it occured
text_fea_dict = dict(zip(list(train_text_features),train_text_fea_counts))


print("Total number of unique words in train data :", len(train_text_features))

Total number of unique words in train data : 4800


In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# building a CountVectorizer with all the words that occured minimum 3 times in train data
text_vectorizer = TfidfVectorizer(min_df=3)
train_text_feature_onehotCoding = text_vectorizer.fit_transform(train['text'])
test_text_feature_onehotCoding = text_vectorizer.transform(test['text'])
# getting all the feature names (words)
train_text_features= text_vectorizer.get_feature_names()

# train_text_feature_onehotCoding.sum(axis=0).A1 will sum every row and returns (1*number of features) vector
train_text_fea_counts = train_text_feature_onehotCoding.sum(axis=0).A1

# zip(list(text_features),text_fea_counts) will zip a word with its number of times it occured
text_fea_dict = dict(zip(list(train_text_features),train_text_fea_counts))


print("Total number of unique words in train data :", len(train_text_features))

Total number of unique words in train data : 4800


In [ ]:
#train['safe_text'] = train['safe_text'].apply(lambda x: re.sub('<[^<]+?>', '', x))
#train['safe_text'] = train['safe_text'].apply(lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",x))
#train['safe_text'] = train['safe_text'].apply(lambda x: re.sub(' +', ' ', x))
#train['safe_text'] = train['safe_text'].apply(lambda x: x.lower())

In [19]:
X = train_text_feature_onehotCoding
X_t = test_text_feature_onehotCoding

In [20]:
df = pd.DataFrame(X.toarray())
df1 = pd.DataFrame(X_t.toarray())

In [21]:
mergeTrain = train.merge(df, left_index=True, right_index=True)
mergeTest = test.merge(df1, left_index=True, right_index=True)


In [22]:
mergeTest

,ID,text,word_count,char_count,avg_word,stopwords,hastags,numerics,0,1,...,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,test_0,rinaldi According new Italian government regul...,18,128,6.166667,5,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,test_1,In 2017 Palestinian judge banned divorce make ...,18,113,5.333333,6,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,test_2,Why explained video take look,10,42,3.666667,4,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,test_3,Ed Davey fasting No contest,7,39,4.714286,1,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,test_4,Is Doja Cat good miss Nicki Minaj,11,48,3.454545,4,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2936,test_2936,want wish Muslim members Congress HAPPY RAMADAN,13,66,4.153846,5,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2937,test_2937,You mean believe conspiracy involving 5G Bill ...,27,166,5.185185,10,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2938,test_2938,Our end persecution animals cruel This Monday ...,16,86,4.437500,5,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2939,test_2939,New York New Jersey California ordered nursing...,18,105,4.888889,5,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train = mergeTrain
test = mergeTest

In [24]:
X = train.drop(['ID','text','target'],axis = 1)
test = test.drop(['ID','text'],axis = 1)
y = train['target']

In [ ]:
#X = np.sqrt(X)
#test = np.sqrt(test)
#from sklearn.preprocessing import MaxAbsScaler
#scaler = MaxAbsScaler()
#X = scaler.fit_transform(X)

In [30]:
X = np.float64(X)
test = np.float64(test)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=30)

In [32]:
from sklearn.metrics import log_loss

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
X_train.shape, X_test.shape

In [33]:
from xgboost import XGBRegressor
xg = XGBRegressor(random_state =34)
xg.fit(X_train, y_train)

# Make predictions
predsxg = xg.predict(X_test)

# Score
print(log_loss(y_test, predsxg))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[15:20:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: 

In [35]:
import catboost as cbt
cbt = cbt.CatBoostRegressor(random_state = 34)
cbt.fit(X_train, y_train)

# Make predictions
predscbt = cbt.predict(X_test)

# Score
print(log_loss(y_test, predscbt))

Learning rate set to 0.051204
0:	learn: 0.4980752	total: 439ms	remaining: 7m 18s
1:	learn: 0.4964530	total: 924ms	remaining: 7m 41s
2:	learn: 0.4949511	total: 1.35s	remaining: 7m 29s
3:	learn: 0.4934931	total: 1.76s	remaining: 7m 18s
4:	learn: 0.4923047	total: 2.19s	remaining: 7m 15s
5:	learn: 0.4905332	total: 2.62s	remaining: 7m 13s
6:	learn: 0.4889449	total: 3.02s	remaining: 7m 8s
7:	learn: 0.4876448	total: 3.46s	remaining: 7m 9s
8:	learn: 0.4865072	total: 3.9s	remaining: 7m 9s
9:	learn: 0.4853598	total: 4.33s	remaining: 7m 8s
10:	learn: 0.4842000	total: 4.75s	remaining: 7m 7s
11:	learn: 0.4835050	total: 5.17s	remaining: 7m 5s
12:	learn: 0.4823078	total: 5.59s	remaining: 7m 4s
13:	learn: 0.4814557	total: 6.02s	remaining: 7m 4s
14:	learn: 0.4802496	total: 6.45s	remaining: 7m 3s
15:	learn: 0.4789545	total: 6.88s	remaining: 7m 3s
16:	learn: 0.4780276	total: 7.29s	remaining: 7m 1s
17:	learn: 0.4770728	total: 7.71s	remaining: 7m
18:	learn: 0.4763784	total: 8.14s	remaining: 7m
19:	learn: 0

157:	learn: 0.4387625	total: 1m 25s	remaining: 7m 37s
158:	learn: 0.4385970	total: 1m 26s	remaining: 7m 37s
159:	learn: 0.4384276	total: 1m 26s	remaining: 7m 36s
160:	learn: 0.4381468	total: 1m 27s	remaining: 7m 36s
161:	learn: 0.4379591	total: 1m 28s	remaining: 7m 35s
162:	learn: 0.4378182	total: 1m 28s	remaining: 7m 35s
163:	learn: 0.4376313	total: 1m 29s	remaining: 7m 34s


KeyboardInterrupt: 

In [32]:
import lightgbm as lgb
lgb = lgb.LGBMRegressor(random_state = 34, n_estimators = 15)
lgb.fit(X_train, y_train)

# Make predictions
predslgm = lgb.predict(X_test)

# Score
print(mean_squared_error(y_test, predslgm)**0.5)

0.6332995801777062


In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingRegressor
seed = 7
#kfold = model_selection.KFold(n_splits = 10, random_state = seed)
model1 = BaggingRegressor(base_estimator = rfc, n_estimators =20,random_state =seed)
model1.fit(X_train, y_train)
preds = model1.predict(X_test)
# Score
print(mean_squared_error(y_test, preds)**0.5)

#results = model_selection.cross_val_score(model1,X,y,cv = kfold)
#print (results.mean())

In [36]:
from sklearn import model_selection
from sklearn.ensemble import BaggingRegressor
seed = 7
#kfold = model_selection.KFold(n_splits = 10, random_state = seed)
model2 = BaggingRegressor(base_estimator = lgb, n_estimators =15,random_state =seed)
model2.fit(X_train, y_train)
preds = model2.predict(X_test)
# Score
print(mean_squared_error(y_test, preds)**0.5)
#results = model_selection.cross_val_score(model1,X,y,cv = kfold)
#print (results.mean())

0.631942228621704


In [37]:
from sklearn import model_selection
from sklearn.ensemble import BaggingRegressor
seed = 7
#kfold = model_selection.KFold(n_splits = 10, random_state = seed)
model4 = BaggingRegressor(base_estimator = cbt,random_state =seed,n_estimators = 101)
model4.fit(X_train, y_train)
preds = model4.predict(X_test)
# Score
print(mean_squared_error(y_test, preds)**0.5)
#results = model_selection.cross_val_score(model1,X,y,cv = kfold)

TypeError: No matching signature found

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingRegressor
seed = 7
#kfold = model_selection.KFold(n_splits = 10, random_state = seed)
model3 = BaggingRegressor(base_estimator = xg, n_estimators =20,random_state =seed)
model3.fit(X_train, y_train)
preds = model3.predict(X_test)
# Score
print(mean_squared_error(y_test, preds)**0.5)
#results = model_selection.cross_val_score(model1,X,y,cv = kfold)
#print (results.mean())

In [ ]:
from sklearn.ensemble import VotingRegressor
estimators = []
estimators.append(('catboost',cbt))
estimators.append(('lgb',lgb))
#estimators.append(('rfc',rfc))
estimators.append(('xgb',xg))
ensemble = VotingRegressor(estimators)
ensemble.fit(X_train, y_train)
preds = ensemble.predict(X_test)
# Score
print(mean_squared_error(y_test, preds)**0.5)

In [ ]:
ens = ensemble.predict(X_test)

#vot =  model1.predict(X_test)
vot2 = model2.predict(X_test)
vot3 = model3.predict(X_test)
vot4 = model4.predict(X_test)
stacked_predictions = np.column_stack((predscbt,ens,vot3,vot2,vot4,predslgm))#predslgm,preds,vot,vot3
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()
meta_model.fit(stacked_predictions,y_test)
final = meta_model.predict(stacked_predictions)
print(mean_squared_error(y_test, final)**0.5)

In [ ]:
testpredcbt = cbt.predict(test)
testpredlgm = lgb.predict(test)
#testpredxgb = xg.predict(test)
testensemble = ensemble.predict(test)
#testvoting = model1.predict(test)
testvoting2 = model2.predict(test)
testvoting3 = model3.predict(test)#testpredlgm,testpredxgb,testvoting,testvoting3
testvoting4 = model4.predict(test)
test_stacked = np.column_stack((testpredcbt,testensemble,testvoting3,testvoting2,testvoting4,testpredlgm))
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()
meta_model.fit(stacked_predictions,y_test)
predictions = meta_model.predict(test_stacked)

In [ ]:
stacked_predictions = np.column_stack((predscbt,predslgm,preds,rfcpreds))
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()
meta_model.fit(stacked_predictions,y_test)
final = meta_model.predict(stacked_predictions)
print(mean_squared_error(y_test, final)**0.5)

In [ ]:
testpredcbt = cbt.predict(test)
testpredlgb = lgb.predict(test)
testpredxgb = xg.predict(test)
testpredrfc = rfc.predict(test)
test_stacked = np.column_stack((testpredcbt,testpredlgb,testpredxgb,testpredrfc))
from sklearn.linear_model import LinearRegression
meta_model = LinearRegression()
meta_model.fit(stacked_predictions,y_test)
predictions = meta_model.predict(test_stacked)

In [ ]:
ss = pd.read_csv('SampleSubmission.csv')
ss.head()

In [ ]:
ss['label'] = predictions
ss.head()

In [ ]:
ss.to_csv('4th_vaccine.csv', index=False)